## Language Analysis of Alexithymic Discourse

<hr>

Alexithymic Language Project / raul@psicobotica.com / V2 release (sept 2020)

<hr>

### N-GRAMs Models

An N-Gram is "A contiguous sequence of N items from a given sample of text or speech".

- Char N-Grams.
- Word N-Grams.
- Language generation with N-Gram models. 

<hr>

[Explanation of N-Gram models](https://en.wikipedia.org/wiki/N-gram)

## Load features dataset
- Data is already pre-processed (1-Preprocessing). 
- Basic NLP features are already calculated (2-Features). 
- Some additional BoW features have been added (3-BoW).
- Some additional TF/IDF features have been added (3-TFIDF).

In [82]:
import pandas as pd 
import numpy as np
import ast
import heapq
import re
import string
import random
import nltk

In [2]:
feats_dataset_path = "https://raw.githubusercontent.com/raul-arrabales/alexithymic-lang/master/data/Prolexitim_v2_features_3.csv"
alex_df = pd.read_csv(feats_dataset_path, header=0, delimiter=";")

In [3]:
alex_df.columns

Index(['Code', 'TAS20', 'F1', 'F2', 'F3', 'Gender', 'Age', 'Card',
       'T_Metaphors', 'T_ToM', 'T_FP', 'T_Interpret', 'T_Desc', 'T_Confussion',
       'Text', 'Alex_A', 'Alex_B', 'Words', 'Sentences', 'Tokens',
       'Tokens_Stop', 'Tokens_Stem_P', 'Tokens_Stem_S', 'POS', 'NER', 'DEP',
       'Lemmas_CNLP', 'Lemmas_Spacy', 'Chars', 'avgWL', 'avgSL', 'Pun_Count',
       'Stop_Count', 'RawTokens', 'Title_Count', 'Upper_Count', 'PRON_Count',
       'DET_Count', 'ADV_Count', 'VERB_Count', 'PROPN_Count', 'NOUN_Count',
       'NUM_Count', 'PUNCT_Count', 'SYM_Count', 'SCONJ_Count', 'CCONJ_Count',
       'INTJ_Count', 'AUX_Count', 'ADP_Count', 'ADJ_Count', 'PRON_Ratio',
       'DET_Ratio', 'ADV_Ratio', 'VERB_Ratio', 'PROPN_Ratio', 'NOUN_Ratio',
       'NUM_Ratio', 'PUNCT_Ratio', 'SYM_Ratio', 'SCONJ_Ratio', 'CCONJ_Ratio',
       'INTJ_Ratio', 'AUX_Ratio', 'ADP_Ratio', 'ADJ_Ratio', 'TTR', 'HTR',
       'BoW_PCA_1', 'BoW_PCA_2', 'BoW_PCA_3', 'TFIDF_PCA_1', 'TFIDF_PCA_2',
       'TFIDF_PCA_3']

In [4]:
alex_df.head()

,Code,TAS20,F1,F2,F3,Gender,Age,Card,T_Metaphors,T_ToM,...,ADP_Ratio,ADJ_Ratio,TTR,HTR,BoW_PCA_1,BoW_PCA_2,BoW_PCA_3,TFIDF_PCA_1,TFIDF_PCA_2,TFIDF_PCA_3
0,bc39e22ca5dba59fbd97c27987878f56,40,16,9,15,2,22,1,0,1,...,0.125000,0.000000,0.562500,0.875000,0.429786,-0.056197,-0.360772,-0.114870,0.168706,0.031455
1,bc39e22ca5dba59fbd97c27987878f56,40,16,9,15,2,22,13HM,0,1,...,0.000000,0.000000,0.857143,1.000000,-0.535592,0.971355,-0.133005,0.867802,0.301337,0.165452
2,20cd825cadb95a71763bad06e142c148,40,12,10,18,2,22,1,0,1,...,0.103448,0.172414,0.344828,0.793103,0.713317,-0.012597,-0.255988,-0.089725,0.143005,0.031664
3,20cd825cadb95a71763bad06e142c148,40,12,10,18,2,22,9VH,0,1,...,0.208333,0.083333,0.458333,0.875000,-0.280320,-0.445467,0.372081,-0.019208,-0.076310,-0.093545
4,20cd825cadb95a71763bad06e142c148,40,12,10,18,2,22,13HM,0,1,...,0.100000,0.200000,0.900000,1.000000,-0.539096,0.998465,-0.135003,0.393093,0.108074,0.043623


## Preparing the corpora
Let's get two corpora, one with "alexithymic language" and the other with "non-alexithymic language". 
- AlexDoc will contain merged text from TAS-20 positive users. 
- NoAlexDoc will contain merged text from TAS-20 negative users. 

In [24]:
# Get all the alexithymic lang together
AlexDoc = ' '.join(alex_df[alex_df.Alex_A == 1].Text)

# Get all non-alexithymic lang together
NoAlexDoc = ' '.join(alex_df[alex_df.Alex_A == 0].Text)

In [25]:
print("As expected, we have a quite unbalanced dataset: Alex: %d; NoAlex: %d." % (len(AlexDoc),len(NoAlexDoc)))

As expected, we have a quite unbalanced dataset: Alex: 10285; NoAlex: 63589.


In [34]:
# AlexDoc

In [31]:
# All to lower case and remove punctuation
AlexDoc = AlexDoc.translate(str.maketrans('', '', string.punctuation)).lower()
NoAlexDoc = NoAlexDoc.translate(str.maketrans('', '', string.punctuation)).lower()

## Creating N-Gram models

### Char N-Gram Model

In [35]:
# Establish the length of the n-grams
char_ngram_length = 6

In [47]:
# Char n-gram calculation function. 
def compute_char_ngrams(doc, length):
    """
    Parameters
    ----------
    doc : str
        Document to extract n-grams from. 
    length : int
        Length of the n-grams (2 is bigram, 3 is a trigram, etc.)
    """
    
    char_ngrams = {}
    
    for i in range(len(doc)-length):  # For each char i in doc
        seq = doc[i:i+length]         # Get current sequence
        # print(seq)
        if seq not in char_ngrams.keys():       # Create entry for new sequences
            char_ngrams[seq] = []                  
        char_ngrams[seq].append(doc[i+length])  # Add new char
    
    return char_ngrams

In [51]:
# Compute char n-grams for the two corpora
alex_char_ngrams = compute_char_ngrams(AlexDoc, char_ngram_length)
noalex_char_ngrams = compute_char_ngrams(NoAlexDoc, char_ngram_length)

In [49]:
alex_char_ngrams.get("niño q")

['u', 'u', 'u', 'u']

In [53]:
noalex_char_ngrams.get("lloran")

['d', 'd', 'd', 'd', 'd']

### Persist the char n-gram models

In [76]:
import json

In [94]:
charNGramAlex_path = "D:\\Dropbox-Array2001\\Dropbox\\DataSets\\Prolexitim-Dataset\\Prolexitim_v2_CharNGram_Alex.json"
charNGramNoAlex_path = "D:\\Dropbox-Array2001\\Dropbox\\DataSets\\Prolexitim-Dataset\\Prolexitim_v2_CharNGram_NoAlex.json"

with open(charNGramAlex_path, 'w', encoding='utf8') as fp:
    json.dump(alex_char_ngrams, fp, ensure_ascii=False)
    
with open(charNGramNoAlex_path, 'w', encoding='utf8') as fp:
    json.dump(noalex_char_ngrams, fp, ensure_ascii=False)

### Predicting text with the Char N-Gram Model

In [60]:
# N-Gram based char generation function. 
def predict_chars(seed, ng_model, ng_length, gen_length):
    """
    Parameters
    ----------
    seed : str
        Initial sequence of chars to generate from. 
    ng_model : dict
        Dictionary with the char N-Gram model. 
    ng_length : int
        Length of the n-grams (2 is bigram, 3 is a trigram, etc.)
    gen_length : int
        Length of the string to be generated.        
    """
    if (len(seed) != ng_length):
        raise Exception("Seed length shoud be equal to N-Grams length.")
    
    generated = seed  # We start with the seed. 

    for i in range(gen_length):
        if seed not in ng_model.keys():
            break
        possible_chars = ng_model[seed]
        next_char = possible_chars[random.randrange(len(possible_chars))]
        generated += next_char
        seed = generated[len(generated)-ng_length:len(generated)] # New seed is now the last sequence.   
    
    return generated

In [69]:
# Speak in "alexithymic language" prediction 
predict_chars("carlit", alex_char_ngrams, char_ngram_length, 200)

'carlitos él tiene que todos juntos y animales nunca habían pasado la noche de pasion con sus amigos y sienten muchas ideas geniales en medio de trabajo una siesta la faena que cercanía y se durmiendo a este'

In [70]:
# Speak in "non-alexithymic language" prediction 
predict_chars("carlit", noalex_char_ngrams, char_ngram_length, 200)

'carlitos con mucho más adecuada para siempre ha terminado un hombre pobre que ahora si quedaba cierta paz y la señores que eso le apetecía nada tocar el violín y tirar flechas con mucha más cercanos el agua'

In [75]:
# Speak in "non-alexithymic language" prediction 
predict_chars("niño q", noalex_char_ngrams, char_ngram_length, 50)

'niño que tiene un gesto un hombre le regalo su abuelitos'

### Word N-Gram Model

In [81]:
# Establish the length of the n-grams
# Let's go for word trigrams
word_ngram_length = 3

In [86]:
# Word n-gram calculation function. 
def compute_word_ngrams(doc, length):
    """
    Parameters
    ----------
    doc : str
        Document to extract n-grams from. 
    length : int
        Length of the n-grams (2 is bigram, 3 is a trigram, etc.)
    """
    ngrams = {}
    
    tokens = nltk.word_tokenize(doc)
    for i in range(len(tokens)-length):
        seq = ' '.join(tokens[i:i+length])
        # print(seq)
        if seq not in ngrams.keys():
            ngrams[seq] = []
        ngrams[seq].append(tokens[i+length])
        
    return ngrams

In [87]:
# Compute word n-grams for the two corpora
alex_word_ngrams = compute_word_ngrams(AlexDoc, word_ngram_length)
noalex_word_ngrams = compute_word_ngrams(NoAlexDoc, word_ngram_length)

In [89]:
alex_word_ngrams.get('largo día de')

['arar']

In [90]:
noalex_word_ngrams.get('largo día de')

['trabajo', 'trabajo']

### Persist the word n-gram models

In [93]:
wordNGramAlex_path = "D:\\Dropbox-Array2001\\Dropbox\\DataSets\\Prolexitim-Dataset\\Prolexitim_v2_WordNGram_Alex.json"
wordNGramNoAlex_path = "D:\\Dropbox-Array2001\\Dropbox\\DataSets\\Prolexitim-Dataset\\Prolexitim_v2_WordNGram_NoAlex.json"

with open(wordNGramAlex_path, 'w', encoding='utf8') as fp:
    json.dump(alex_word_ngrams, fp, ensure_ascii=False)
    
with open(wordNGramNoAlex_path, 'w', encoding='utf8') as fp:
    json.dump(noalex_word_ngrams, fp, ensure_ascii=False)

### Predicting text with the Word N-Gram Model

In [99]:
# N-Gram based word generation function. 
def predict_words(seed, ng_model, ng_length, gen_length):
    """
    Parameters
    ----------
    seed : str
        Initial sequence of words to generate from. 
    ng_model : dict
        Dictionary with the word N-Gram model. 
    ng_length : int
        Length of the n-grams (2 is bigram, 3 is a trigram, etc.)
    gen_length : int
        Length of the string to be generated (in words).        
    """
    seed_words = nltk.word_tokenize(seed)
    
    if (len(seed_words) != ng_length):
        raise Exception("Seed length shoud be equal to N-Grams length.")
    
    generated = seed
    for i in range(gen_length):
        if seed not in ng_model.keys():
            break
        possible_words = ng_model[seed]
        next_word = possible_words[random.randrange(len(possible_words))]
        generated += ' ' + next_word
        seq_words = nltk.word_tokenize(generated)
        seed = ' '.join(seq_words[len(seq_words)-ng_length:len(seq_words)])
    
    return generated

In [105]:
# Generate alexithymic text
predict_words('largo día de', alex_word_ngrams, word_ngram_length, 50)

'largo día de arar la tierra un dia una persona estaba caminando por un bosque cercano a su hogar y ha llegado a un lugar maravilloso donde la mano del hombre nunca había llegado la naturaleza se habia encargado de poner todo en su sitio así todos todos los elementos interactúaban y cooperaban'

In [107]:
# Generate non-alexithymic text
predict_words('largo día de', noalex_word_ngrams, word_ngram_length, 50)

'largo día de trabajo llegó la plácida siesta mereció la pena aguantar un poco más y saborear ese delicioso cochinillo que con tanto cariño le había reglado su abuelo se ponía una gran presión sobre sus hombros más de la que su propio abuelo pretendía con aquel presente ¿cómo si quiera podría iniciarse'

In [110]:
predict_words('un niño que', noalex_word_ngrams, word_ngram_length, 100)

'un niño que deseaba con todas sus fuerzas tocar el violín pero sus padres le obligaban a estudiar una carrera con más salidas un grupo de exploradores que iban a hacer un recorrido por una gran montaña pero se les hizo de noche y tuvieron que acampar todos juntos encima del césped una cascada en mitad de la naturaleza donde todos los animales cuando se acercaba el buen tiempo se dirigían para estar horas y horas tocando el violin estaba deseoso de poder salir a jugar un grupo de trabajadores que tras la jornada de trabajo se reencontraban con sus sueños los que'

In [111]:
predict_words('un niño que', noalex_word_ngrams, word_ngram_length, 100)

'un niño que le gustaba descubrir cosas y decidió subir por unas escaleras que descubrió cuando sus padres fueron a un bar se adentró en un mundo mágico en el que lo pasamos muy bien esta era la historía de un señor anciano en el que habitaban los animales mas extraños que os podríais imaginar pero todos ellos tenían una característica común todos ellos estaban privados del sentido de la vista porque no tenían ojos esta particularidad les había llevado a desarrollar sus otros sentidos lo que a priori nos podría parecer un handicap tenía también algunas ventajas como por ejemplo que como'

In [114]:
predict_words('un niño que', alex_word_ngrams, word_ngram_length, 100)

'un niño que quería tocar una guitarra pero le regalaron un violín estaba tumbado con sus amigos viajando en un tren destino a lo desconocido en su viaje en elefante tenía que cruzar una gran catarata fue lento y tortuoso pero al final lo consiguió la acababa de conocer en una discoteca y habían pasado la noche juntos y ahí estaba ella muerta por un ataque al corazón y el sin saber que hacer… un niño al que no le gustaba la música pero sus padres le obligaban a tocar el violin un padre y sus hijos deecansando un dia de fiesta en'